In [ ]:
import numpy as np
from astropy.io import fits as fits
import matplotlib.pyplot as plt
import matplotlib.patches as patches 
from astropy.coordinates import Angle, SkyCoord
from astropy.coordinates import match_coordinates_sky, search_around_sky, match_coordinates_3d
from astropy import units as u
import matplotlib.cm as cm
from astropy.nddata import Cutout2D
import astropy.visualization as viz
from astropy.wcs import WCS
from astropy.visualization import ZScaleInterval
import astropy.coordinates as coords
from astropy.visualization.wcsaxes import WCSAxes
import astropy.visualization.wcsaxes.frame as frame
from astropy.table import Table
import pandas as pd
import scipy.stats as st
from scipy.stats import kde, sigmaclip
import time
import pickle
import os
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, Normalizer
from IPython.display import Image
import seaborn as sns

%matplotlib notebook
%matplotlib inline

#### Getting both catalogs

In [ ]:
Chandra_Data = Table.read('Chandra_COSMOS_Legacy_20151120_4d.fits', format='fits').to_pandas()
HSC_Data = pd.read_csv("/Users/linaflorez/Downloads/156674.csv.gz", dtype = np.float64).dropna(axis = 0, how = "any")

#### Making a pickle from the HSC data release 2 Ultra Deep catalog

In [ ]:
# HSC_catalog = SkyCoord(ra = HSC_Data.ra.values * u.degree, dec = HSC_Data.dec.values * u.degree)
# with open('pdr2_ud_skycoords.pkl', 'wb') as f:
#     pickle.dump(HSC_catalog, f)

#### Accessing the pickles

In [ ]:
#pickles contain the ra & dec in a more manageable way!
with open('/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Analysis/pdr2_ud_skycoords.pkl', 'rb') as f:
    HSC_catalog = pickle.load(f)
with open('/Users/linaflorez/Desktop/ObsCos/COSMOS_research/venv/Chandra_skycoords.pkl', 'rb') as f:
    Chandra_catalog = pickle.load(f)

#### Applying the match_coordinates_sky function -- aka crossmatching

In [ ]:
#idx and sep2d are the length of the Chandra catalog and index the HSC catalog
idx, separation_angles, dist3d = match_coordinates_sky(Chandra_catalog, HSC_catalog, nthneighbor = 1)

#### Making a dataframe with all of the matches sources

In [ ]:
# Combining both arrays
result_df = pd.concat([Chandra_Data.reset_index(), HSC_Data.iloc[idx].reset_index()], axis = 1)
result_df.rename(columns={"RA_x": "Chandra_ra", "DEC_x": "Chandra_dec", "ra":"HSC_ra", "dec":"HEC_dec"},inplace = True)

# Maximum separation between sources
max_sep = 2.5 * u.arcsec

# A counterpart is determined based on the separation angle from it's match_coordinates_sky output
no_matches_df = result_df[separation_angles > max_sep]
matches_df = result_df[separation_angles < max_sep]

print("Size of HSC Catalog:", len(HSC_Data))
print("Number of Matches:", len(matches_df))
print("Number of sources without a counterpart:", len(no_matches_df))

#### Making a scatterplot

In [ ]:
plt.style.use("seaborn-whitegrid")
plt.figure(figsize = (10,10))
plt.scatter(HSC_Data.ra[::50], HSC_Data.dec[::50], color = "tomato", label = "All HSC Sources \n(Every 50th point)")
plt.plot(matches_df.Chandra_ra, matches_df.Chandra_dec, ".", color = "darkblue", markersize = 15,label = "Matches")
plt.xlim(148, 153)
plt.xlabel("RA (degrees)", weight = "bold", size = 20)
plt.ylabel("DEC (degrees)", weight = "bold", size = 20)
plt.legend(loc = "best", prop = {"size":15})
plt.tick_params("both", labelsize = 20)
plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/HSC_v_matches.png", transparent = True, dpi = 300)

### Let's look at sources without counterparts

In [ ]:
plt.style.use("seaborn-whitegrid")
plt.figure(figsize = (10,10))
plt.scatter(Chandra_Data.RA_x, Chandra_Data.DEC_x, color = "tomato", label = "All Chandra Sources")
plt.plot(no_matches_df.Chandra_ra, no_matches_df.Chandra_dec, ".", color = "darkblue", markersize = 15,label = "No Matches: %i" % (len(no_matches_df)))
plt.xlabel("RA (degrees)", weight = "bold", size = 20)
plt.ylabel("DEC (degrees)", weight = "bold", size = 20)
plt.legend(loc = "best", prop = {"size":15})
plt.tick_params("both", labelsize = 20)
plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/match_v_nomatch.png", transparent = True)

In [ ]:
no_matches_df.rename(columns={"Chandra_ra":"ra", "Chandra_dec": "dec"},inplace = True)
no_matches_df[["ra","dec"]].to_csv("ud_nomatch_hscmap_input.csv")
no_matches_df.rename(columns={"ra":"Chandra_ra", "dec":"Chandra_dec"},inplace = True)
no_matches_df.reset_index(drop=True, inplace=True)
no_matches_df.drop(columns='index')

#### Taking hscmap output and prepping the files for my stacking function

In [ ]:
matches_df.columns
matches_df.rename(columns={"Chandra_ra":"ra", "Chandra_dec": "dec"},inplace = True)
gcmodel_mag24ish = matches_df.loc[abs(24 - matches_df["g_cmodel_mag"]) < 0.05][['ra','dec']][10:20]
gcmodel_mag24ish.shape
gcmodel_mag24ish.to_csv("gcmodel_mag24ish.csv", index=False) #going to input this into hscmap

#### Doing the stacking

In [ ]:
def cutout_func(band, result, cut_size): 
    
    dimensions = cut_size * 6
    num = np.zeros((dimensions, dimensions))
    denom = np.zeros((dimensions, dimensions))

    iterating_through_fits_files = np.linspace(0, len(result)-1, len(result), dtype = "int")
    
    if len(iterating_through_fits_files) == 34:
        iterating_through_fits_files = np.delete(iterating_through_fits_files, [2],0)
    
    
    for i in iterating_through_fits_files:
        
        #opening FITS file
        respective_band = result[band].iloc[i]
        hdulist = fits.open(respective_band)
        flux_mag0 = hdulist[0].header["FLUXMAG0"]

        # hscMap output
        image, mask, variance = hdulist[1].data, hdulist[2].data, hdulist[3].data 

        #big general cutout to make sure all stacked images are the same dimensions
        wcs = WCS(hdulist[1].header)
        if "Chandra_ra" in result:
            RA, DEC = result["Chandra_ra"].iloc[i], result["Chandra_dec"].iloc[i]
        else:
            RA, DEC = result["ra"].iloc[i], result["dec"].iloc[i]
            
        coords = SkyCoord(ra=RA*u.degree, dec=DEC*u.degree)
        cut_center, size = SkyCoord(coords), cut_size
        cutout_size = np.array([size, size]) * u.arcsec
        general_cutout = Cutout2D(image, cut_center, cutout_size, wcs=wcs)
        variance_of_cutout = Cutout2D(variance, cut_center, cutout_size, wcs=wcs).data

        #added on/stacked to initialized num/denom arrays
        num += general_cutout.data/variance_of_cutout 
        denom += 1/variance_of_cutout
        

    cutouts = num/denom
    variance_stacked_image = denom**-1
    sky_subtracted =  cutouts - np.median(cutouts)

    #Determining magnitude
    center = (int((6*cut_size)/2)-3, int((6*cut_size)/2)+3)
    original_magnitude = -2.5 * np.log10(np.sum(cutouts[center[0]:center[1], center[0]:center[1]])/flux_mag0)
    sky_subtracted_magnitude = -2.5 * np.log10(np.sum(sky_subtracted[center[0]:center[1], center[0]:center[1]])/flux_mag0)

    #upper limit calculation
    sigma_flux =  np.sum(variance_stacked_image[center[0]:center[1], center[0]:center[1]])**(1/2)
    average_center = np.sum(sky_subtracted[center[0]:center[1], center[0]:center[1]])
    upper_lim = -2.5 * np.log10((average_center + (2*sigma_flux))/flux_mag0)
    ave = -2.5 * np.log10((average_center)/flux_mag0)
    lower_lim = -2.5 * np.log10((average_center - (2*sigma_flux))/flux_mag0)
    
    upper_lim2 = -2.5 * np.log10(((2*sigma_flux))/flux_mag0)
    print("Upper lim: %.3f, Average: %.3f, Lower lim: %.3f, Upperlim2: %.3f" % (upper_lim, ave, lower_lim, upper_lim2))
    

    # Plotting stacked image!
    plt.style.use("dark_background")
    fig,((ax1, ax2,ax3)) = plt.subplots(1,3, figsize = (40,10))
    
    title1 = ('Original Stacking %i %s-band images \n Central 1" region: %.3f mag' % (iterating_through_fits_files.shape[0], band[0], original_magnitude))
    ax1.set_title(title1, weight = "bold", size = 20)
    plot1 = ax1.imshow(cutouts, cmap = "plasma", extent=[-size/2 ,size/2,-size/2 ,size/2], aspect='auto')
    ax1.contour(cutouts, 3, extent=[-size/2 ,size/2,-size/2 ,size/2],cmap='Greys')
    ax1.set_xlabel("Arcsecs", weight = "bold", size = 20)
    ax1.set_ylabel("Arcsecs", weight = "bold", size = 20)
    ax1.tick_params("both", labelsize = 20)
    cbar1 = fig.colorbar(plot1, label = "Flux Units", ax = ax1)
    cbar1.ax.tick_params(labelsize=20, size=15)
    

    title2 = ('Sky Subtracted Stack of %i %s-band images \n Central 1" region: %.3f mag (Upper Limit)' % (iterating_through_fits_files.shape[0], band[0], upper_lim))
    ax2.set_title(title2, weight = "bold", size = 20)
    plot2 = ax2.imshow(sky_subtracted, cmap = "plasma", extent=[-size/2 ,size/2,-size/2 ,size/2], aspect='auto')
    ax2.contour(sky_subtracted, 3, extent=[-size/2, size/2, -size/2, size/2],cmap='Greys')
    ax2.set_xlabel("Arcsecs", weight = "bold", size = 20)
    ax2.set_ylabel("Arcsecs", weight = "bold", size = 20)
    ax2.tick_params("both", labelsize = 20)
    cbar2 = fig.colorbar(plot2, label = "Flux Units", ax = ax2)
    cbar2.ax.tick_params(labelsize=20, size=15)
    
    
    ax3.set_title("Sky subtracted stack", weight = "bold", size = 20)
    ax3.hist(sky_subtracted, bins = np.arange(-.1, .1, 0.002))
    ax3.set_xlim(-0.01, 0.04)
    ax3.set_xlabel("Flux Units", weight = "bold", size = 20)
    ax3.tick_params("both", labelsize = 20)

    plt.tight_layout()
    plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/%s_cutout.png" % (band), transparent = True)

    return 

In [ ]:
def stacking_fits_files(folder_path, size):

    fits_dec, g_fits, r_fits, i_fits, z_fits, y_fits= [], [], [], [], [], []
    fits_files_df = pd.DataFrame()

    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            if file.startswith("cutout-HSC-G"):
                fits_dec.append(float(os.path.join(subdir, file).split(os.sep)[-2]))
                g_fits.append(os.path.join(subdir, file)) 

            if file.startswith("cutout-HSC-R"):
                r_fits.append(os.path.join(subdir, file))

            if file.startswith("cutout-HSC-I"):
                i_fits.append(os.path.join(subdir, file))

            if file.startswith("cutout-HSC-Z"):
                z_fits.append(os.path.join(subdir, file))

            if file.startswith("cutout-HSC-Y"):
                y_fits.append(os.path.join(subdir, file))


    fits_files_df = pd.DataFrame()
    fits_files_df["declination"], fits_files_df["gfits"] = fits_dec, g_fits
    fits_files_df["rfits"], fits_files_df["ifits"], fits_files_df["zfits"], fits_files_df["yfits"] = r_fits, i_fits, z_fits, y_fits


    indices = []
    
    if folder_path == "/Users/linaflorez/Downloads/FITS_FILES/":
        for i in fits_files_df.declination:
            indices.append(no_matches_df[no_matches_df.Chandra_dec == i].index.values[0])
        testing_stacking = pd.concat([no_matches_df.iloc[indices].reset_index(), fits_files_df], axis = 1)

    else:
        for i in fits_files_df.declination:
            indices.append(matches_df[matches_df.dec == i].index.values[0])
        testing_stacking = pd.concat([matches_df.loc[indices].reset_index(), fits_files_df], axis = 1)

    cutout_func("gfits", testing_stacking, size)
    cutout_func("rfits", testing_stacking, size)
    cutout_func("ifits", testing_stacking, size)
    cutout_func("zfits", testing_stacking, size)
    cutout_func("yfits", testing_stacking, size)
    
    
    return testing_stacking

In [ ]:
stacking_fits_files("/Users/linaflorez/Downloads/psf24mag_withcounterparts/", 9)

In [ ]:
dataframe = stacking_fits_files("/Users/linaflorez/Downloads/FITS_FILES/", 9)

In [ ]:
dataframe.rename(columns={"Chandra_ra":"ra", "Chandra_dec": "dec"},inplace = True)
dataframe = dataframe.drop([2])
dataframe[["ra","dec"]].to_csv("sources_wo_counterparts.csv", index=False)
dataframe

In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize = (35,15))

min_matches_df = matches_df.loc[(matches_df.g_cmodel_magsigma < 0.2) & (matches_df.r_cmodel_magsigma < 0.2) & (matches_df.i_cmodel_magsigma < 0.2)]
sns.kdeplot(min_matches_df["g_cmodel_mag"] - min_matches_df["r_cmodel_mag"], min_matches_df["r_cmodel_mag"] - min_matches_df["i_cmodel_mag"],\
            cmap='Reds', ax = ax1, shade=True).set(xlim = (-0.5, 2), ylim = (-.5, 2))

min_HSC_Data = HSC_Data.loc[(HSC_Data.g_cmodel_magsigma < 0.2) & (HSC_Data.r_cmodel_magsigma < 0.2) & (HSC_Data.i_cmodel_magsigma < 0.2)]
sns.kdeplot((min_HSC_Data["g_cmodel_mag"] - min_HSC_Data["r_cmodel_mag"])[0::50],(min_HSC_Data["r_cmodel_mag"] - min_HSC_Data["i_cmodel_mag"])[0::50],\
                cmap='Blues' , ax = ax2, shade=True).set(xlim = (-0.5, 2), ylim = (-.5, 2))


visual_counterpart = sns.kdeplot(pd.g_psfflux_mag - pd.r_psfflux_mag, pd.r_psfflux_mag - pd.i_psfflux_mag, cmap='Reds', shade=True, ax = ax3).set(xlim = (-0.5, 2), ylim = (-.5, 2))



ax1.set_xlabel("(g-r)$_{cmodel\:matches}$", weight = "bold", size = 20)
ax1.set_ylabel("(r-i)$_{cmodel\:matches}$", weight = "bold", size = 20)

ax2.set_xlabel("(g-r)$_{cmodel\:HSC\:ultradeep}$", weight = "bold", size = 20)
ax2.set_ylabel("(r-i)$_{cmodel\:HSC\:ultradeep}$", weight = "bold", size = 20)

ax3.set_xlabel("(g-r)$_{confirmed\:visual\:counterparts}$", weight = "bold", size = 20)
ax3.set_ylabel("(r-i)$_{confirmed\:visual\:counterparts}$", weight = "bold", size = 20)

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)
ax3.tick_params("both", labelsize = 20)

plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/Matches/gr_ri.png", transparent = "True")


In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (20,8))
min_matches_df = matches_df.loc[(matches_df.g_psfflux_magsigma < 0.2) & (matches_df.r_psfflux_magsigma < 0.2) & (matches_df.i_psfflux_magsigma < 0.2)]
sns.kdeplot(min_matches_df["g_psfflux_mag"] - min_matches_df["r_psfflux_mag"], min_matches_df["r_psfflux_mag"] - min_matches_df["i_psfflux_mag"],\
            cmap='Reds', ax = ax1, shade=True).set(xlim = (-0.5, 2), ylim = (-.5, 2))

min_HSC_Data = HSC_Data.loc[(HSC_Data.g_psfflux_magsigma < 0.2) & (HSC_Data.r_psfflux_magsigma < 0.2) & (HSC_Data.i_psfflux_magsigma < 0.2)]
sns.kdeplot((min_HSC_Data["g_psfflux_mag"] - min_HSC_Data["r_psfflux_mag"])[0::50],(min_HSC_Data["r_psfflux_mag"] - min_HSC_Data["i_psfflux_mag"])[0::50],\
                cmap='Blues' , ax = ax2, shade=True).set(xlim = (-0.5, 2), ylim = (-.5, 2))
ax1.set_xlabel("(g-r)$_{psf\:matches}$", weight = "bold", size = 20)
ax1.set_ylabel("(r-i)$_{psf\:matches}$", weight = "bold", size = 20)

ax2.set_xlabel("(g-r)$_{psf\:HSC\:ultradeep}$", weight = "bold", size = 20)
ax2.set_ylabel("(r-i)$_{psf\:HSC\:ultradeep}$", weight = "bold", size = 20)

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)
plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/Matches/gr_ri_psf.png", transparent = "True")

In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (20,8))

min_matches_df = matches_df.loc[(matches_df.g_cmodel_magsigma < 0.2) & (matches_df.r_cmodel_magsigma < 0.2)]
sns.kdeplot(min_matches_df["g_cmodel_mag"] - min_matches_df["r_cmodel_mag"], min_matches_df["flux_F"],\
            cmap='Reds', ax = ax1, shade=True).set(xlim = (-0.5, 2), ylim = (-3e-15, 3e-14))

min_matches_df = matches_df.loc[(matches_df.g_psfflux_magsigma < 0.2) & (matches_df.r_psfflux_magsigma < 0.2)]
sns.kdeplot((min_matches_df["g_psfflux_mag"] - min_matches_df["r_psfflux_mag"]),min_matches_df["flux_F"],\
                cmap='Blues', ax = ax2, shade=True).set(xlim = (-0.5, 2), ylim = (-3e-15, 3e-14))

ax1.set_xlabel("(g-r)$_{cmodel\:matches}$", weight = "bold", size = 20)
ax1.set_ylabel("flux$_{F_{matches}(0.5 - 10keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax2.set_xlabel("(g-r)$_{psf\:matches}$", weight = "bold", size = 20)
ax2.set_ylabel("flux$_{F_{matches}(0.5 - 10keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)

plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/gr_fluxF.png", transparent = True)


In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (20,8))

min_matches_df = matches_df.loc[(matches_df.r_cmodel_magsigma < 0.2) & (matches_df.i_cmodel_magsigma < 0.2)]
sns.kdeplot(min_matches_df["r_cmodel_mag"] - min_matches_df["i_cmodel_mag"], min_matches_df["flux_F"],\
            cmap='Reds', ax = ax1, shade=True).set(xlim = (-0.5, 2), ylim = (-3e-15, 3e-14))

min_matches_df = matches_df.loc[(matches_df.g_psfflux_magsigma < 0.2) & (matches_df.r_psfflux_magsigma < 0.2)]
sns.kdeplot((min_matches_df["r_psfflux_mag"] - min_matches_df["i_psfflux_mag"]),min_matches_df["flux_F"],\
                cmap='Blues', ax = ax2, shade=True).set(xlim = (-0.5, 2), ylim = (-3e-15, 3e-14))

ax1.set_xlabel("(r-i)$_{cmodel\:matches}$", weight = "bold", size = 20)
ax1.set_ylabel("flux$_{F_{matches}(0.5 - 10keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax2.set_xlabel("(r-i)$_{psf\:matches}$", weight = "bold", size = 20)
ax2.set_ylabel("flux$_{F_{matches}(0.5 - 10keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)

plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/ri_fluxF.png", transparent = True)


In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (20,8))

min_matches_df = matches_df.loc[(matches_df.r_cmodel_magsigma < 0.2) & (matches_df.i_cmodel_magsigma < 0.2)]
sns.kdeplot(min_matches_df["g_cmodel_mag"] - min_matches_df["r_cmodel_mag"], min_matches_df["flux_S"],\
            cmap='Reds', ax = ax1, shade=True).set(xlim = (-0.5, 2), ylim = (-5e-16, 1e-14))


min_matches_df = matches_df.loc[(matches_df.g_psfflux_magsigma < 0.2) & (matches_df.r_psfflux_magsigma < 0.2)]
sns.kdeplot((min_matches_df["g_psfflux_mag"] - min_matches_df["r_psfflux_mag"]),min_matches_df["flux_S"],\
                cmap='Blues', ax = ax2, shade=True).set(xlim = (-0.5, 2), ylim = (-5e-16, 1e-14))

ax1.set_xlabel("(g-r)$_{cmodel\:matches}$", weight = "bold", size = 20)
ax1.set_ylabel("flux$_{S_{matches}(0.5 - 2keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax2.set_xlabel("(g-r)$_{psf\:matches}$", weight = "bold", size = 20)
ax2.set_ylabel("flux$_{S_{matches}(0.5 - 2keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)

plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/gr_fluxS.png", transparent = True)


In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (20,8))

min_matches_df = matches_df.loc[(matches_df.r_cmodel_magsigma < 0.2) & (matches_df.i_cmodel_magsigma < 0.2)]
sns.kdeplot(min_matches_df["r_cmodel_mag"] - min_matches_df["i_cmodel_mag"], min_matches_df["flux_S"],\
            cmap='Reds', ax = ax1, shade=True).set(xlim = (-0.5, 2), ylim = (-5e-16, 1e-14))


min_matches_df = matches_df.loc[(matches_df.r_psfflux_magsigma < 0.2) & (matches_df.i_psfflux_magsigma < 0.2)]
sns.kdeplot((min_matches_df["r_psfflux_mag"] - min_matches_df["i_psfflux_mag"]),min_matches_df["flux_S"],\
                cmap='Blues', ax = ax2, shade=True).set(xlim = (-0.5, 2), ylim = (-5e-16, 1e-14))

ax1.set_xlabel("(r-i)$_{cmodel\:matches}$", weight = "bold", size = 20)
ax1.set_ylabel("flux$_{S_{matches}(0.5 - 2keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax2.set_xlabel("(r-i)$_{psf\:matches}$", weight = "bold", size = 20)
ax2.set_ylabel("flux$_{S_{matches}(0.5 - 2keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)

plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/ri_fluxS.png", transparent = True)


In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (20,8))
min_matches_df = matches_df.loc[(matches_df.g_cmodel_magsigma < 0.2) & (matches_df.r_cmodel_magsigma < 0.2)]
sns.kdeplot(min_matches_df["g_cmodel_mag"] - min_matches_df["r_cmodel_mag"], min_matches_df["flux_H"],\
            cmap='Reds', ax = ax1, shade=True).set(xlim = (-0.5, 2), ylim = (-2e-15, 1.5e-14))


min_matches_df = matches_df.loc[(matches_df.g_psfflux_magsigma < 0.2) & (matches_df.r_psfflux_magsigma < 0.2)]
sns.kdeplot((min_matches_df["g_psfflux_mag"] - min_matches_df["r_psfflux_mag"]),min_matches_df["flux_H"],\
                cmap='Blues', ax = ax2, shade=True).set(xlim = (-0.5, 2), ylim = (-2e-15, 1.5e-14))

ax1.set_xlabel("(g-r)$_{cmodel\:matches}$", weight = "bold", size = 20)
ax1.set_ylabel("flux$_{H_{matches}(2 - 10keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax2.set_xlabel("(g-r)$_{psf\:matches}$", weight = "bold", size = 20)
ax2.set_ylabel("flux$_{H_{matches}(2 - 10keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)

plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/gr_fluxH.png", transparent = True)


In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (20,8))

min_matches_df = matches_df.loc[(matches_df.r_cmodel_magsigma < 0.2) & (matches_df.i_cmodel_magsigma < 0.2)]
sns.kdeplot(min_matches_df["r_cmodel_mag"] - min_matches_df["i_cmodel_mag"], min_matches_df["flux_H"],\
            cmap='Reds', ax = ax1, shade=True).set(xlim = (-0.5, 2), ylim = (-2e-15, 1.5e-14))

min_matches_df = matches_df.loc[(matches_df.r_psfflux_magsigma < 0.2) & (matches_df.i_psfflux_magsigma < 0.2)]
sns.kdeplot((min_matches_df["r_psfflux_mag"] - min_matches_df["i_psfflux_mag"]),min_matches_df["flux_H"],\
                cmap='Blues', ax = ax2, shade=True).set(xlim = (-0.5, 2), ylim = (-2e-15, 1.5e-14))

ax1.set_xlabel("(r-i)$_{cmodel\:matches}$", weight = "bold", size = 20)
ax1.set_ylabel("flux$_{H_{matches}(2 - 10keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax2.set_xlabel("(r-i)$_{psf\:matches}$", weight = "bold", size = 20)
ax2.set_ylabel("flux$_{H_{matches}(2 - 10keV [erg*cm^{-2}*s^{-1}])}$", weight = "bold", size = 20)

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)

plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/ri_fluxH.png", transparent = True)


In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15,5))
hist = sns.distplot(no_counterparts.flux_F, color = "tomato", rug = True, bins = np.arange(0,1e-13,1e-15), \
                    kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax1)
hist.set_xlabel("flux_F${_{no\:counterparts}}$", fontsize=15, fontweight='bold')
hist.set_ylabel("Counts", fontsize=15, fontweight='bold')
hist.set_xlim([0,.15e-13])
# hist.set_yscale("log")
ax1.tick_params("both", labelsize = 15)

hist2 = sns.distplot(min_matches_df.flux_F, color = "tomato", rug = True, bins = np.arange(0,1e-13,1e-15), \
                    kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax2)
hist2.set_xlabel("flux_F${_{matches}}(0.5 - 10keV\:[erg*cm^{-2}*s^{-1}])$", fontsize=15, fontweight='bold')
hist2.set_ylabel("Counts", fontsize=15, fontweight='bold')
hist2.set_xlim([0,.15e-13])
ax2.tick_params("both", labelsize = 15)

hist3 = sns.distplot(Chandra_Data.flux_F, color = "magenta", rug = False,bins = np.arange(0,1e-13,0.1e-14),\
                     kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax3)
# hist2.set_yscale("log")
hist3.set_xlim([0,1.5e-14])
hist3.set_xlabel("flux_F${_{Chandra\:Data}}}$", fontsize=15, fontweight='bold')
hist3.set_ylabel("Counts", fontsize=15, fontweight='bold')
ax3.tick_params("both", labelsize = 15)
plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/Comparison/flux_F.png", transparent = True)

In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15,5))
hist = sns.distplot(no_counterparts.flux_S, color = "tomato", rug = True, bins = np.arange(0,1e-13,1e-15), \
                    kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax1)
hist.set_xlabel("flux_S${_{no\:counterparts}}$", fontsize=15, fontweight='bold')
hist.set_ylabel("Counts", fontsize=15, fontweight='bold')
hist.set_xlim([0,.5e-14])
ax1.tick_params("both", labelsize = 15)

hist2 = sns.distplot(matches_df.flux_S, color = "tomato", rug = True, bins = np.arange(0,1e-13,1e-15), \
                    kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax2)
hist2.set_xlabel("flux_S${_{matches}}(0.5 - 2keV\:[erg*cm^{-2}*s^{-1}])$", fontsize=15, fontweight='bold')
hist2.set_ylabel("Counts", fontsize=15, fontweight='bold')
hist2.set_xlim([0,.5e-14])
ax2.tick_params("both", labelsize = 15)

hist3 = sns.distplot(Chandra_Data.flux_S, color = "magenta", rug = False,bins = np.arange(0,1e-13,0.1e-15),\
                     kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax3)
# hist2.set_yscale("log")
hist3.set_xlim([0,.5e-14])
hist3.set_xlabel("flux_S${_{Chandra\:Data}}}$", fontsize=15, fontweight='bold')
hist3.set_ylabel("Counts", fontsize=15, fontweight='bold')
ax3.tick_params("both", labelsize = 15)
plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/Comparison/flux_S.png", transparent = True)

In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15,5))
hist = sns.distplot(no_counterparts.flux_H, color = "tomato", rug = True, bins = np.arange(0,1e-13,1e-15), \
                    kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax1)
hist.set_xlabel("flux_H${_{no\:counterparts}}$", fontsize=15, fontweight='bold')
hist.set_ylabel("Counts", fontsize=15, fontweight='bold')
hist.set_xlim([0,1e-14])
ax1.tick_params("both", labelsize = 15)

hist2 = sns.distplot(matches_df.flux_H, color = "tomato", rug = True, bins = np.arange(0,1e-13,1e-15), \
                    kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax2)
hist2.set_xlabel("flux_H${_{matches}}(2 - 10keV\:[erg*cm^{-2}*s^{-1}])$", fontsize=15, fontweight='bold')
hist2.set_ylabel("Counts", fontsize=15, fontweight='bold')
hist2.set_xlim([0,1e-14])
ax2.tick_params("both", labelsize = 15)

hist3 = sns.distplot(Chandra_Data.flux_H, color = "magenta", rug = False,bins = np.arange(0,1e-13,0.1e-15),\
                     kde = False, norm_hist = False, hist_kws={'edgecolor':'black'},ax = ax3)
# hist2.set_yscale("log")
hist3.set_xlim([0,1e-14])
hist3.set_xlabel("flux_H${_{Chandra\:Data}}}$", fontsize=15, fontweight='bold')
hist3.set_ylabel("Counts", fontsize=15, fontweight='bold')
ax3.tick_params("both", labelsize = 15)
plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/Comparison/flux_H.png", transparent = True)

In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15,5))


hist2 = sns.distplot(matches_df.HR[matches_df.HR > -60], color = "blue", bins = np.arange(-1,1,.05), \
                     rug = False, hist=True, kde=True, hist_kws={'edgecolor':'black'}, ax = ax1)
hist2.set_xlabel("HR$_{matched}$", fontsize=15, fontweight='bold')
hist2.set_ylabel("Density", fontsize=15, fontweight='bold')


hist = sns.distplot(min_nocounterparts_df.HR[min_nocounterparts_df.HR > -20], color = "tomato", bins = np.arange(-1,1,.05), \
                    rug = True, hist=True, kde=True,hist_kws={'edgecolor':'black'}, ax = ax2)
hist.set_xlabel("HR$_{not\:matched}$", fontsize=15, fontweight='bold')
hist.set_ylabel("Density", fontsize=15, fontweight='bold')
ax1.tick_params("both", labelsize = 20)


hist3 = sns.distplot(Chandra_Data.HR[Chandra_Data.HR > -60], color = "magenta", bins = np.arange(-1,1,.05), \
                     rug = False, hist=True, kde=True, hist_kws={'edgecolor':'black'}, ax = ax3)
hist3.set_xlabel("HR$_{Chandra\:Data}$", fontsize=15, fontweight='bold')
hist3.set_ylabel("Density", fontsize=15, fontweight='bold')

ax1.tick_params("both", labelsize = 20)
ax2.tick_params("both", labelsize = 20)
ax3.tick_params("both", labelsize = 20)

plt.tight_layout()
plt.savefig("/Users/linaflorez/Desktop/ObsCos/COSMOS_research/Plots/Comparison/HR.png", transparent = True)

